<a href="https://colab.research.google.com/github/garmartirosy/GithubActions/blob/main/Commodity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymrio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 5.5 MB/s eta 0:00:00
  Created wheel for pymrio: filename=pymrio-0.5.4-py3-none-any.whl size=140078 sha256=3774da5213c3c165a0881386924b64884abac183211b1b6bb1aa9cea216ead7d
  Stored in directory: /root/.cache/pip/wheels/f7/a2/98/de741b07dff4ebf8f79d17cd943f9d64f77c4fb2f1e08b7816
Successfully built pymrio
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 3.1.5
    Uninstalling openpyxl-3.1.5:
      Successfully uninstalled openpyxl-3.1.5


In [ ]:
import pymrio

pymrio.download_exiobase3(storage_folder='content', system='pxp', years=[2012])


exio_data2 = pymrio.parse_exiobase3(path="/content/content/IOT_2012_pxp.zip")


In [ ]:
# Step 1: Extract unique commodities and create the Commodity Table
commodities = exio_data.get_index('product')
commodity_df = pd.DataFrame({'CommodityCode': commodities, 'CommodityName': commodities})
print("Initial Commodity Table:")
print(commodity_df.head())
print(f"Number of unique commodities: {len(commodity_df)}")

# Step 2: Sum final demand data for each commodity across all regions
# Extract the second level (commodity) from the MultiIndex in final demand
final_demand = exio_data.Y.groupby(level=1).sum().sum(axis=1)

# Map final demand to Commodity Table
# Extract only the commodity name from CommodityCode for mapping
commodity_df['CommodityNameOnly'] = commodity_df['CommodityCode'].apply(lambda x: x[1])
commodity_df['TotalFinalDemand'] = commodity_df['CommodityNameOnly'].map(final_demand.to_dict())

# Fill missing values with 0
commodity_df['TotalFinalDemand'] = commodity_df['TotalFinalDemand'].fillna(0)

# Drop intermediate column (optional)
commodity_df = commodity_df.drop(columns=['CommodityNameOnly'])

print("Commodity Table with Final Demand:")
print(commodity_df.head())

# Step 3: Add Region Names (same as before)
region_mapping = pd.read_csv('/content/exio_country_names.csv')  # Replace with actual filename
region_mapping = region_mapping.rename(columns={'CountryCode': 'RegionCode'})
commodity_df['RegionCode'] = commodity_df['CommodityCode'].apply(lambda x: x[0])
commodity_df = pd.merge(commodity_df, region_mapping, how='left', left_on='RegionCode', right_on='RegionCode')
commodity_df = commodity_df.drop(columns=['RegionCode'])

print("Final Commodity Table with Region Names:")
print(commodity_df.head())

# Step 4: Save the Final Table
commodity_df.to_csv("CommodityTable_with_RegionNames.csv", index=False)
print("Commodity Table saved to 'CommodityTable_with_RegionNames.csv'.")

commodity_df.head()

Initial Commodity Table:
                                            CommodityCode  \
AT Paddy rice                            (AT, Paddy rice)   
   Wheat                                      (AT, Wheat)   
   Cereal grains nec              (AT, Cereal grains nec)   
   Vegetables, fruit, nuts  (AT, Vegetables, fruit, nuts)   
   Oil seeds                              (AT, Oil seeds)   

                                            CommodityName  
AT Paddy rice                            (AT, Paddy rice)  
   Wheat                                      (AT, Wheat)  
   Cereal grains nec              (AT, Cereal grains nec)  
   Vegetables, fruit, nuts  (AT, Vegetables, fruit, nuts)  
   Oil seeds                              (AT, Oil seeds)  
Number of unique commodities: 9800
Commodity Table with Final Demand:
                                            CommodityCode  \
AT Paddy rice                            (AT, Paddy rice)   
   Wheat                                      (AT, Wheat

In [ ]:

commodity_df.head()

,CommodityCode,CommodityName,TotalFinalDemand,Country
0,"(AT, Paddy rice)","(AT, Paddy rice)",42468.666360,Austria
1,"(AT, Wheat)","(AT, Wheat)",52164.332401,Austria
2,"(AT, Cereal grains nec)","(AT, Cereal grains nec)",64604.979864,Austria
3,"(AT, Vegetables, fruit, nuts)","(AT, Vegetables, fruit, nuts)",564548.697571,Austria
4,"(AT, Oil seeds)","(AT, Oil seeds)",21319.398558,Austria


In [ ]:
exio_data.satellite.F_Y # Stressors/impacts of the final demand


region                                                                                      AT  \
category                                           Final consumption expenditure by households   
stressor                                                                                         
Taxes less subsidies on products purchased: Total                                          0.0   
Other net taxes on production                                                              0.0   
Compensation of employees; wages, salaries, & e...                                         0.0   
Compensation of employees; wages, salaries, & e...                                         0.0   
Compensation of employees; wages, salaries, & e...                                         0.0   
...                                                                                        ...   
Energy Carrier Net TMAR                                                                    0.0   
Energy Carrier Net TOTH                                                                    0.0   
Energy Carrier Net TRAI                                                                    0.0   
Energy Carrier Net TROA                                                               131692.0   
Energy Carrier Net LOSS                                                                    0.0   

region                                                                                                                                   \
category                                           Final consumption expenditure by non-profit organisations serving households (NPISH)   
stressor                                                                                                                                  
Taxes less subsidies on products purchased: Total                                                 0.0                                     
Other net taxes on production                                                                     0.0                                     
Compensation of employees; wages, salaries, & e...                                                0.0                                     
Compensation of employees; wages, salaries, & e...                                                0.0                                     
Compensation of employees; wages, salaries, & e...                                                0.0                                     
...                                                                                               ...                                     
Energy Carrier Net TMAR                                                                           0.0                                     
Energy Carrier Net TOTH                                                                           0.0                                     
Energy Carrier Net TRAI                                                                           0.0                                     
Energy Carrier Net TROA                                                                           0.0                                     
Energy Carrier Net LOSS                                                                           0.0                                     

region                                                                                          \
category                                           Final consumption expenditure by government   
stressor                                                                                         
Taxes less subsidies on products purchased: Total                                          0.0   
Other net taxes on production                                                              0.0   
Compensation of employees; wages, salaries, & e...                                         0.0   
Compensation of employees; wages, salaries, & e...                                         0

In [ ]:
exio_data.satellite.S # Direct stressor/impact coefficients


In [ ]:
exio_data.satellite.S_Y # Stressor/impact coefficients of the final demand


In [ ]:
exio_data.satellite.M # MRIO extension multipliers (total requirement factors of consumption)


region                                                     AT            \
sector                                             Paddy rice     Wheat   
stressor                                                                  
Taxes less subsidies on products purchased: Total           0  0.038572   
Other net taxes on production                               0 -0.029868   
Compensation of employees; wages, salaries, & e...          0  0.012633   
Compensation of employees; wages, salaries, & e...          0  0.114603   
Compensation of employees; wages, salaries, & e...          0  0.071593   
...                                                       ...       ...   
Energy Carrier Net TMAR                                     0  0.187353   
Energy Carrier Net TOTH                                     0  0.500155   
Energy Carrier Net TRAI                                     0  0.016087   
Energy Carrier Net TROA                                     0  4.572679   
Energy Carrier Net LOSS                                     0  1.297262   

region                                                                \
sector                                             Cereal grains nec   
stressor                                                               
Taxes less subsidies on products purchased: Total           0.031191   
Other net taxes on production                              -0.028553   
Compensation of employees; wages, salaries, & e...          0.010061   
Compensation of employees; wages, salaries, & e...          0.091947   
Compensation of employees; wages, salaries, & e...          0.056601   
...                                                              ...   
Energy Carrier Net TMAR                                     0.076280   
Energy Carrier Net TOTH                                     0.111207   
Energy Carrier Net TRAI                                     0.011314   
Energy Carrier Net TROA                                     0.433099   
Energy Carrier Net LOSS                                     0.980251   

region                                                                      \
sector                                             Vegetables, fruit, nuts   
stressor                                                                     
Taxes less subsidies on products purchased: Total                 0.022326   
Other net taxes on production                                    -0.023541   
Compensation of employees; wages, salaries, & e...                0.011485   
Compensation of employees; wages, salaries, & e...                0.100745   
Compensation of employees; wages, salaries, & e...                0.065578   
...                                                                    ...   
Energy Carrier Net TMAR                                           0.082434   
Energy Carrier Net TOTH                                           0.050655   
Energy Carrier Net TRAI                                           0.008410   
Energy Carrier Net TROA                                           0.673631   
Energy Carrier Net LOSS                                           0.770327   

region                                                        \
sector                                             Oil seeds   
stressor                                                       
Taxes less subsidies on products purchased: Total   0.020752   
Other net taxes on production                      -0.032973   
Compensation of employees; wages, salaries, & e...  0.007567   
Compensation of employees; wages, salaries, & e...  0.066963   
Compensation of employees; wages, salaries, & e...  0.042972   
...                                                      ...   
Energy Carrier Net TMAR                             0.126071   
Energy Carrier Net TOTH                             1.153899   
Energy Carrier Net TRAI                             0.013061   
Energy Carrier Net TROA                             0.682931   
Energy Carrier 

In [ ]:
exio_data.satellite.D_cba # Consumption based accounts per sector


In [ ]:
exio_data.satellite.D_pba # Production based accounts per sector


In [ ]:
exio_data.satellite.D_cba_reg # Consumption based accounts per region


,AT,BE,BG,CY,CZ,DE,DK,EE,ES,FI,...,TR,TW,NO,ID,ZA,WA,WL,WE,WF,WM
stressor,,,,,,,,,,,,,,,,,,,,,
Taxes less subsidies on products purchased: Total,15139.157282,21604.837801,2317.260011,739.962300,8093.163680,1.162161e+05,13594.613424,1264.282825,4.027786e+04,11377.269452,...,5.805907e+04,1.238815e+04,17460.015630,4.440107e+04,2.562962e+04,2.856662e+05,3.163803e+05,3.663393e+04,2.199720e+05,3.313399e+05
Other net taxes on production,6630.981437,6594.077438,596.721094,802.870852,892.716510,1.972135e+04,3618.679728,208.158378,1.597786e+04,851.385566,...,8.589797e+03,1.145181e+04,1889.344090,1.013251e+04,7.194075e+03,7.468433e+04,7.781110e+04,1.115059e+04,5.310754e+04,8.021299e+04
"Compensation of employees; wages, salaries, & employers' social contributions: Low-skilled",10600.334700,14328.472086,1475.059339,643.952191,3814.394865,6.662234e+04,12256.219492,675.368783,4.482648e+04,7951.025651,...,2.622563e+04,1.833204e+04,14089.606620,3.009314e+04,1.357018e+04,1.286771e+05,7.056219e+04,1.761032e+04,9.602737e+04,9.133921e+04
"Compensation of employees; wages, salaries, & employers' social contributions: Medium-skilled",69853.058248,83749.046051,9295.556247,4016.929479,29566.979419,5.625299e+05,58836.452602,3912.459353,2.242330e+05,45967.959507,...,1.438003e+05,1.084638e+05,84837.545973,1.748294e+05,4.985289e+04,8.020209e+05,5.946280e+05,9.416484e+04,2.604289e+05,5.454570e+05
"Compensation of employees; wages, salaries, & employers' social contributions: High-skilled",77003.951343,104815.026580,8887.745935,4602.810867,38918.001639,7.151998e+05,67803.084385,4839.857047,2.307542e+05,55641.335076,...,1.185428e+05,5.489409e+04,90366.900335,6.641509e+04,7.490366e+04,4.150997e+05,3.840695e+05,7.976293e+04,2.616010e+05,4.590403e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Energy Carrier Net TMAR,27475.161047,56166.147014,5712.333288,49171.450263,11094.856881,5.049973e+05,250613.593258,2336.933099,9.696348e+04,46924.590664,...,1.647301e+05,9.052772e+04,233782.413019,2.383679e+05,8.676308e+03,2.610897e+05,1.391471e+05,3.088541e+04,9.322015e+04,1.797892e+05
Energy Carrier Net TOTH,30552.140767,25064.865901,8671.857533,935.640936,25226.547221,1.883539e+05,9233.141085,3376.546951,5.169226e+04,12455.320088,...,3.177876e+04,1.456539e+04,13108.523136,1.624224e+04,6.099116e+03,1.279707e+05,8.694868e+04,7.889984e+04,5.793361e+04,5.506378e+04
Energy Carrier Net TRAI,15211.105827,9044.159864,1982.222810,343.287076,8881.586241,7.367858e+04,4219.534442,1045.112711,3.018407e+04,5813.303210,...,1.296181e+04,7.912645e+03,5288.903610,3.853652e+03,1.360228e+04,7.368860e+04,2.553540e+04,2.920218e+04,2.455077e+04,2.568386e+04


In [ ]:
exio_data.satellite.D_pba_reg # Production based accounts per region


In [ ]:
exio_data.satellite.D_imp_reg # Import accounts per region


In [ ]:
exio_data.satellite.D_exp_reg # Export accounts per region


In [ ]:
Factor (includes FactorID and FlowUUID)
Industry (5-char)
IndustryFactor
Commodity (6-char)
CommodityFactor
ImportIndustryFactor
ImportCommodityFactor
ImportContributions
ImportMultiplierFactor


